In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [104]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [105]:
train_cat = train.select_dtypes('object')
test_cat = test.select_dtypes('object')
missing = (100 * (train_cat.isna().sum())/len(train)).sort_values(ascending=False)
large_missing_vals = [col for col in missing.index if missing[col] > 30]
large_missing_vals

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu']

In [106]:
train_cat = train_cat.drop(large_missing_vals, axis=1)
tes_cat = test_cat.drop(large_missing_vals, axis=1)

In [111]:
cat = ["MSZoning", "Utilities","BldgType","Heating","KitchenQual","SaleCondition","LandSlope"]

In [109]:
train_cat = train_cat[cat]
#test_cat = test_cat[[cat]]

,MSZoning,Utilities,BldgType,Heating,KitchenQual,SaleCondition,LandSlope
0,RL,AllPub,1Fam,GasA,Gd,Normal,Gtl
1,RL,AllPub,1Fam,GasA,TA,Normal,Gtl
2,RL,AllPub,1Fam,GasA,Gd,Normal,Gtl
3,RL,AllPub,1Fam,GasA,Gd,Abnorml,Gtl
4,RL,AllPub,1Fam,GasA,Gd,Normal,Gtl
...,...,...,...,...,...,...,...
1455,RL,AllPub,1Fam,GasA,TA,Normal,Gtl
1456,RL,AllPub,1Fam,GasA,TA,Normal,Gtl
1457,RL,AllPub,1Fam,GasA,Gd,Normal,Gtl
1458,RL,AllPub,1Fam,GasA,Gd,Normal,Gtl


In [63]:
for col in train_cat.columns:
    train_cat[col] = train_cat[col].fillna(train_cat[col].mode()[0])

In [71]:
for col in test_cat.columns:
    test_cat[col] = test_cat[col].fillna(test_cat[col].mode()[0])

In [72]:
train_cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


In [74]:
target_variable = train['SalePrice']

In [75]:
train_cat = pd.get_dummies(train_cat, dtype='int')
train_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0


In [76]:
test_cat = pd.get_dummies(test_cat, dtype='int')
test_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0


In [57]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier
rf = RandomForestClassifier()

# Fit the classifier to your data
rf.fit(train_cat, target_variable)

# Get feature importances
feature_importances = rf.feature_importances_

# Select important features based on a threshold or top k features
feature_importances_df = pd.DataFrame(feature_importances, index = train_cat.columns, columns=['Importance'])
feature_importances_df.sort_values(by='Importance', ascending=False).head(15)

ValueError: could not convert string to float: 'RL'

In [35]:
from sklearn.decomposition import PCA

# Initialize PCA
pca = PCA(n_components=10)  # You can adjust the number of components as needed

# Fit and transform PCA on your data
train_cat_pca = pca.fit_transform(train_cat)

# Get the names of the principal components
pc_names = [f"PC{i}" for i in range(1, pca.n_components_ + 1)]
pc_names 
loadings = pd.DataFrame(pca.components_, columns=train_cat.columns)

# Display the loadings for the first few principal components
loadings.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.005139,0.032156,-0.005461,0.049847,-0.071403,-0.002061,0.002061,0.079907,0.011092,0.002987,...,0.000156,0.076327,-0.000648,-0.061267,-0.017983,-0.001424,-0.002944,-0.004217,-0.049956,0.076524
1,0.005909,0.055376,0.010050,-0.278269,0.206933,-0.001803,0.001803,-0.215503,-0.001754,0.000501,...,0.002962,0.027800,-0.001035,-0.020345,0.003916,0.000989,-0.000568,0.001630,-0.033947,0.027980
2,0.005690,-0.000264,0.001636,-0.010577,0.003515,0.001610,-0.001610,-0.280310,-0.019625,-0.001294,...,0.000455,0.151848,0.001876,-0.175476,0.024344,0.000893,0.002507,0.004508,-0.187363,0.155111
3,0.005198,-0.050473,-0.001091,0.078512,-0.032146,0.001135,-0.001135,0.373770,0.020114,0.007225,...,0.001196,0.085051,-0.001300,-0.092341,0.019406,0.001690,-0.002267,0.000855,-0.106129,0.086446
4,0.005899,0.055808,0.007751,-0.176915,0.107457,0.007673,-0.007673,-0.001402,0.011807,0.000172,...,-0.001417,0.024504,0.002428,-0.027342,-0.001564,0.000676,0.011582,0.000120,-0.033703,0.022889


In [49]:
from sklearn.feature_selection import SelectKBest, chi2

# Initialize SelectKBest with a scoring function (e.g., chi-square)
kbest = SelectKBest(score_func=chi2, k=10)  # You can adjust k as needed

# Fit and transform the selector on your data
train_cat_selected = kbest.fit_transform(train_cat, target_variable)

# Get the indices of selected features
selected_indices = kbest.get_support(indices=True)

# Get the names of selected features
selected_features = train_cat.columns[selected_indices]
selected_features

ValueError: could not convert string to float: 'RL'